## Artificial Neural Networks

In [2]:
# Import torch
import torch

### Neural Networks

The network and the shape of layers and weights is shown below.

![image info](https://assets.datacamp.com/production/repositories/4094/datasets/90a76cfc0248297aa65f7cb9bdc17602b6b1d84b/net-ex.jpg)

In [3]:
input_layer = torch.tensor([-1.,-2.5,1.2,1.4])
weight_1 = torch.rand(4,4)
weight_2 = torch.rand(4,4)
weight_3 = torch.rand(4,4)

# Calculate the first and second hidden layer
hidden_1 = torch.matmul(input_layer, weight_1)
hidden_2 = torch.matmul(hidden_1, weight_2)

# Calculate the output
print(torch.matmul(hidden_2, weight_3))

# Calculate weight_composed_1 and weight
weight_composed_1 = torch.matmul(weight_1, weight_2)
weight = torch.matmul(weight_composed_1, weight_3)

# Multiply input_layer with weight
print(torch.matmul(input_layer, weight))

tensor([-2.6004, -2.5051, -1.7203, -2.0068])
tensor([-2.6004, -2.5051, -1.7203, -2.0068])


### ReLU activation
We are going to build a neural network which has non-linearity

In [4]:
#Import NN
from torch import nn

# Instantiate non-linearity
relu = nn.ReLU()

# Apply non-linearity on the hidden layers
hidden_1_activated = relu(torch.matmul(input_layer, weight_1))
hidden_2_activated = relu(torch.matmul(hidden_1_activated, weight_2))
print(torch.matmul(hidden_2_activated, weight_3))

# Apply non-linearity in the product of first two weights. 
weight_composed_1_activated = relu(torch.matmul(weight_1, weight_2))

# Multiply `weight_composed_1_activated` with `weight_3
weight = torch.matmul(weight_composed_1_activated, weight_3)

# Multiply input_layer with weight
print(torch.matmul(input_layer, weight))

tensor([0.7950, 0.9726, 0.5402, 0.3354])
tensor([-2.6004, -2.5051, -1.7203, -2.0068])


As expected, the results are different from the one without relu.

This time we are going to have a different number of units in the layers of the neural network. The input layer will still have 4 features, but then the first hidden layer will have 6 units and the output layer will have 2 units.

![image info](https://assets.datacamp.com/production/repositories/4094/datasets/d55fe04c7d4cc7e2b1614e39e77e832ce56c3fc8/net-ex2.jpg)

In [5]:
# Instantiate ReLU activation function as relu
relu = nn.ReLU()

# Initialize weight_1 and weight_2 with random numbers
weight_1 = torch.rand(4, 6)
weight_2 = torch.rand(6, 2)

# Multiply input_layer with weight_1
hidden_1 = torch.matmul(input_layer, weight_1)

# Apply ReLU activation function over hidden_1 and multiply with weight_2
hidden_1_activated = relu(hidden_1)
print(torch.matmul(hidden_1_activated, weight_2))

tensor([0., 0.])


### Calculating loss function in PyTorch

In [6]:
# Initialize the scores and ground truth
logits = torch.tensor([[-1.2, 0.12, 4.8]])
ground_truth = torch.tensor([2])

# Instantiate cross entropy loss
criterion = nn.CrossEntropyLoss()

# Compute and print the loss
loss = criterion(logits,ground_truth)
print(loss)

tensor(0.0117)
